In [1]:
# using ollama
%pip install -U langchain-ollama

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [18]:
import os
from langchain_community.chat_models import ChatOllama
from langchain.schema import SystemMessage, HumanMessage

In [27]:
problem_statement = """
The system being developed is called the Parking Lot System. The system consists of two main
interfaces – The Administrator console, which is useful at the Parking Lot Owner’s end, and
The User console, which is useful at the parking lot. The Administrator console provides a
central management interface for the parking lots of an owner. It enables the owner to add
and remove parking lots, analyse the parking lot usage and set parking prices all in a single
place. The system expects the Owner to set prices by himself based on the usage information
provided to him by the system. The User console enables the users of the parking lots to
estimate parking charges and if the user is ready to park, confirm their entry in the parking.
The User console computes the total parking charge for a particular user and shows it at his /
her exit. It also allows the user to pay the parking charges through a payment gateway. The
user console communicates internally with the
Administrator console over an internet connection to provide real-time usage analysis. It also
consumes any pricing changes carried out from the Administrator console and shows the

pricing accordingly to new incoming users. The main objective of the Parking Lot System is to
provide a cost-effective solution for the management of multiple parking lots in a city through
various features like real-time utilization reports and allowance for dynamic price setting of
parking spaces. There are multiple applications of the system:
a. Dynamic pricing based on utilization: In the current scenario the owner suffers from
dwindling revenues even under high utilization scenarios. This is basically due to the
mostly fixed pricing even during peak hours. The system would allow the owner to
dynamically set prices depending on the usage of the parking lots in turn leading to
better revenues.
b. Removing Human Intervention: In the current scenario, the parking lots are operated
by humans who issue parking tokens to the users of the parking lots and then collect
the dues at the exit. They also are tasked with ensuring there is enough space to fill
the new cars. However, the system intends to remove these managers as the system
itself would record the details of the cars parked and then compute and collect dues
at the exit maintaining logs during the whole process."""

### Identify Classes

In [28]:
import ollama
import json
import re

def identify_classes(problem_statement):
    """
    Identifies potential class names from a given problem statement using Ollama LLaMA 3.2.
    """
    prompt = f"""
    You are an expert software analyst. Your task is to extract potential class names from a given problem statement. 
    Follow this step-by-step approach:

    1. Identify the key **nouns** representing domain concepts.
    2. If a noun represents a role or system entity, classify it as a **class**.
    3. Ignore verbs and adjectives.
    4. **Output only valid JSON** in the following format without any explanation:
       {{
           "classes": ["Class1", "Class2", "Class3"]
       }}

    **Problem Statement:**
    {problem_statement}

    Now extract the class names based on the above rules.  
    **Do NOT include any explanation — output JSON only.**
    """
      
    # Generate response using Ollama LLaMA 3.2
    response = ollama.chat(
        model="llama3.2",
        messages=[{"role": "user", "content": prompt}],
        options={
            "temperature": 0,
            "top_p": 1,
            "top_k": 1
        }
    )
    
    # Debugging: Print raw response
    response_text = response['message']['content'].strip()
    print("Raw Response from LLaMA 3.2:\n", response_text)

    # ✅ Extract JSON part only using regex
    json_match = re.search(r"\{.*\}", response_text, re.DOTALL)
    if json_match:
        cleaned_text = json_match.group(0).strip()
    else:
        print("Error: No valid JSON found in response.")
        return []

    # Additional Debugging: Print cleaned text before parsing
    print("Cleaned JSON:\n", repr(cleaned_text))

    # ✅ Check if cleaned_text is empty
    if not cleaned_text:
        print("Error: Cleaned JSON is empty. Cannot parse.")
        return []

    # ✅ Parse JSON safely
    try:
        output = json.loads(cleaned_text)  # Convert string to JSON
        return output.get("classes", [])  # Ensure output is a list
    except json.JSONDecodeError as e:
        print("JSON parsing error:", str(e))
        return []

identified_classes = identify_classes(problem_statement)
print("Identified Classes:", identified_classes)


Raw Response from LLaMA 3.2:
 {
  "classes": [
    "Administrator",
    "User",
    "ParkingLotOwner",
    "ParkingLot",
    "PaymentGateway"
  ]
}
Cleaned JSON:
 '{\n  "classes": [\n    "Administrator",\n    "User",\n    "ParkingLotOwner",\n    "ParkingLot",\n    "PaymentGateway"\n  ]\n}'
Identified Classes: ['Administrator', 'User', 'ParkingLotOwner', 'ParkingLot', 'PaymentGateway']


parking_lot_output

Raw Response from LLaMA 3.2:
 {
  "classes": [
    "Administrator",
    "User",
    "ParkingLotOwner",
    "ParkingLot",
    "PaymentGateway"
  ]
}
Cleaned JSON:
 '{\n  "classes": [\n    "Administrator",\n    "User",\n    "ParkingLotOwner",\n    "ParkingLot",\n    "PaymentGateway"\n  ]\n}'
Identified Classes: ['Administrator', 'User', 'ParkingLotOwner', 'ParkingLot', 'PaymentGateway']

metro_station_output

Raw Response from LLaMA 3.2:
 {
  "classes": [
    "MetroStation",
    "TicketDistributor",
    "Traveller",
    "Passenger",
    "Child",
    "Adult",
    "MetroPass",
    "TimeCard",
    "Transaction",
    "User",
    "Machine",
    "TouchScreen",
    "CurrencyNote"
  ]
}
Cleaned JSON:
 '{\n  "classes": [\n    "MetroStation",\n    "TicketDistributor",\n    "Traveller",\n    "Passenger",\n    "Child",\n    "Adult",\n    "MetroPass",\n    "TimeCard",\n    "Transaction",\n    "User",\n    "Machine",\n    "TouchScreen",\n    "CurrencyNote"\n  ]\n}'
Identified Classes: ['MetroStation', 'TicketDistributor', 'Traveller', 'Passenger', 'Child', 'Adult', 'MetroPass', 'TimeCard', 'Transaction', 'User', 'Machine', 'TouchScreen', 'CurrencyNote']

### Identify Attributes

In [29]:
import ollama
import json
import re

def identify_attributes(problem_statement, classes):
    """
    Identifies attributes for each class extracted from the problem statement using Ollama LLaMA 3.2.
    Ensures deterministic output and structured JSON format.
    """
    prompt = f"""
    You are an expert software analyst. Your task is to extract **attributes** for each identified class from the given problem statement.

    **Rules to Identify Attributes:**
    1. Attributes describe **properties or characteristics** of a class.
    2. Ignore verbs, complex entities, and relationships.
    3. Common attributes include **names, IDs, types, and statuses**.
    4. **Output ONLY valid JSON** in the following format (without any explanation):
       {{
           "attributes": {{
               "Class1": ["attribute1", "attribute2"],
               "Class2": ["attribute1", "attribute2"]
           }}
       }}

    **Classes Identified:**
    {classes}

    **Problem Statement:**
    {problem_statement}

    Now extract the attributes for each class.
    **Do NOT include any explanation — output JSON only.**
    """

    # Generate response using Ollama LLaMA 3.2
    response = ollama.chat(
        model="llama3.2",
        messages=[{"role": "user", "content": prompt}],
        options={
            "temperature": 0,
            "top_p": 1,
            "top_k": 1
        }
    )
    
    # ✅ Debugging: Print raw response
    response_text = response['message']['content'].strip()
    print("Raw Response from LLaMA 3.2:\n", response_text)

    # ✅ Extract JSON part only using regex
    json_match = re.search(r"\{.*\}", response_text, re.DOTALL)
    if json_match:
        cleaned_text = json_match.group(0).strip()
    else:
        print("Error: No valid JSON found in response.")
        return {}

    # ✅ Additional Debugging: Print cleaned text before parsing
    print("Cleaned JSON:\n", repr(cleaned_text))

    # ✅ Check if cleaned_text is empty
    if not cleaned_text:
        print("Error: Cleaned JSON is empty. Cannot parse.")
        return {}

    # ✅ Parse JSON safely
    try:
        output = json.loads(cleaned_text)  # Convert string to JSON
        return output.get("attributes", {})  # Extract attributes dictionary
    except json.JSONDecodeError as e:
        print("JSON parsing error:", str(e))
        return {}

identified_attributes = identify_attributes(problem_statement, identified_classes)

print("\nExtracted Attributes:\n", identified_attributes)


Raw Response from LLaMA 3.2:
 {
  "attributes": {
    "Administrator": ["ID", "Type", "Status"],
    "User": ["ID", "Type", "Status"],
    "ParkingLotOwner": ["Name", "ID", "Type", "Status"],
    "ParkingLot": ["Name", "ID", "Type", "Status"],
    "PaymentGateway": ["Name", "ID", "Type", "Status"]
  }
}
Cleaned JSON:
 '{\n  "attributes": {\n    "Administrator": ["ID", "Type", "Status"],\n    "User": ["ID", "Type", "Status"],\n    "ParkingLotOwner": ["Name", "ID", "Type", "Status"],\n    "ParkingLot": ["Name", "ID", "Type", "Status"],\n    "PaymentGateway": ["Name", "ID", "Type", "Status"]\n  }\n}'

Extracted Attributes:
 {'Administrator': ['ID', 'Type', 'Status'], 'User': ['ID', 'Type', 'Status'], 'ParkingLotOwner': ['Name', 'ID', 'Type', 'Status'], 'ParkingLot': ['Name', 'ID', 'Type', 'Status'], 'PaymentGateway': ['Name', 'ID', 'Type', 'Status']}


parking_lot_output

Raw Response from LLaMA 3.2:
 {
  "attributes": {
    "Administrator": ["ID", "Type", "Status"],
    "User": ["ID", "Type", "Status"],
    "ParkingLotOwner": ["Name", "ID", "Type", "Status"],
    "ParkingLot": ["Name", "ID", "Type", "Status"],
    "PaymentGateway": ["Name", "ID", "Type", "Status"]
  }
}
Cleaned JSON:
 '{\n  "attributes": {\n    "Administrator": ["ID", "Type", "Status"],\n    "User": ["ID", "Type", "Status"],\n    "ParkingLotOwner": ["Name", "ID", "Type", "Status"],\n    "ParkingLot": ["Name", "ID", "Type", "Status"],\n    "PaymentGateway": ["Name", "ID", "Type", "Status"]\n  }\n}'

Extracted Attributes:
 {'Administrator': ['ID', 'Type', 'Status'], 'User': ['ID', 'Type', 'Status'], 'ParkingLotOwner': ['Name', 'ID', 'Type', 'Status'], 'ParkingLot': ['Name', 'ID', 'Type', 'Status'], 'PaymentGateway': ['Name', 'ID', 'Type', 'Status']}

metro_station_output

Raw Response from LLaMA 3.2:
 {
  "attributes": {
    "MetroStation": ["ID", "name"],
    "TicketDistributor": ["ID", "type", "status", "capacity"],
    "Traveller": ["ID", "age", "type"],
    "Passenger": ["ID", "type"],
    "Child": ["ID", "type"],
    "Adult": ["ID", "type"],
    "MetroPass": ["ID", "type", "duration"],
    "TimeCard": ["ID", "type", "duration"],
    "Transaction": ["ID", "status", "amount", "payment_method"],
    "User": ["ID", "name", "registered_number"],
    "Machine": ["ID", "type", "touch_screen"],
    "TouchScreen": ["ID", "type"],
    "CurrencyNote": ["ID", "type"]
  }
}
Cleaned JSON:
 '{\n  "attributes": {\n    "MetroStation": ["ID", "name"],\n    "TicketDistributor": ["ID", "type", "status", "capacity"],\n    "Traveller": ["ID", "age", "type"],\n    "Passenger": ["ID", "type"],\n    "Child": ["ID", "type"],\n    "Adult": ["ID", "type"],\n    "MetroPass": ["ID", "type", "duration"],\n    "TimeCard": ["ID", "type", "duration"],\n    "Transaction": ["ID", "status", "amount", "payment_method"],\n    "User": ["ID", "name", "registered_number"],\n    "Machine": ["ID", "type", "touch_screen"],\n    "TouchScreen": ["ID", "type"],\n    "CurrencyNote": ["ID", "type"]\n  }\n}'

Extracted Attributes:
 {'MetroStation': ['ID', 'name'], 'TicketDistributor': ['ID', 'type', 'status', 'capacity'], 'Traveller': ['ID', 'age', 'type'], 'Passenger': ['ID', 'type'], 'Child': ['ID', 'type'], 'Adult': ['ID', 'type'], 'MetroPass': ['ID', 'type', 'duration'], 'TimeCard': ['ID', 'type', 'duration'], 'Transaction': ['ID', 'status', 'amount', 'payment_method'], 'User': ['ID', 'name', 'registered_number'], 'Machine': ['ID', 'type', 'touch_screen'], 'TouchScreen': ['ID', 'type'], 'CurrencyNote': ['ID', 'type']}

### Identify Operations

In [30]:
import json
import re
import ollama

def identified_operations(problem_statement, classes):
    """
    Identifies potential operations for each class from a given problem statement using LLaMA 3.2 (Ollama) with CoT prompting.
    """
    prompt = f"""
    You are an expert software analyst. Your task is to identify **operations** from the given problem statement and their respective classes. 
    Follow this detailed step-by-step process carefully to ensure accurate results:

    ### **Step-by-Step Approach:**
    1. **Identify Operations for Each Class:**
       - Carefully analyze the problem statement for any **verbs** or **actions** linked to each class.
       - If a verb is associated with an object or role, treat it as a candidate for a class operation.
       - Ensure that operations reflect the **core behavior** or responsibility of the class.
       - Format each operation as a function/method name in **camelCase**.
       - Ignore vague or irrelevant actions.

    2. **Ensure Coherence Between Classes and Operations:**
       - If two or more classes are logically related, ensure their operations are consistent.
       - Make sure that the operations are relevant to the class context.
    3. **Ignore Unrelated or Redundant Terms:**
       - Ignore adjectives, adverbs, and irrelevant terms unless they provide meaningful context.
       - Focus on meaningful, domain-relevant terms only.
    4. **Output ONLY valid JSON** in the following format (without any explanation):
    ### **Output Format (Strict JSON):**
    Return only valid JSON in the following format:
    {{
        "operations": {{
            "Class1": ["operation1", "operation2"],
            "Class2": ["operation1", "operation2"],
            ...
        }}
    }}

    **Classes Identified:**
    {classes}

    **Problem Statement:**
    {problem_statement}

    Now extract the operations for each class.
    **Do NOT include any explanation — output JSON only.**
    """
    
   # Generate response using Ollama LLaMA 3.2
    response = ollama.chat(
        model="llama3.2",
        messages=[{"role": "user", "content": prompt}],
        options={
            "temperature": 0,
            "top_p": 1,
            "top_k": 1
        }
    )
    
    # ✅ Debugging: Print raw response
    response_text = response['message']['content'].strip()
    print("Raw Response from LLaMA 3.2:\n", response_text)

    # Clean JSON response (remove backticks and formatting)
    cleaned_text = re.sub(r"```json|```", "", response_text).strip()

    # Additional Debugging: Print cleaned text before parsing
    print("Cleaned JSON:\n", repr(cleaned_text))

    # Check if cleaned_text is empty
    if not cleaned_text:
        print("Error: Cleaned JSON is empty. Cannot parse.")
        return {}

    # Parse JSON safely
    try:
        output = json.loads(cleaned_text)  # Convert string to JSON
        return output.get("operations", {})  
    except json.JSONDecodeError as e:
        print("JSON parsing error:", str(e))
        return {}

# Get identified operations
identified_operations = identified_operations(problem_statement, identified_classes)

# Print result
print("\nIdentified Operations:", identified_operations)


Raw Response from LLaMA 3.2:
 {
  "operations": {
    "Administrator": ["addParkingLot", "removeParkingLot", "analyseUsage", "setPrice"],
    "User": ["estimateCharge", "confirmEntry", "payCharges", "getTotalCharge"],
    "ParkingLotOwner": ["setPrices", "allowDynamicPricing"],
    "ParkingLot": [],
    "PaymentGateway": ["processPayment"]
  }
}
Cleaned JSON:
 '{\n  "operations": {\n    "Administrator": ["addParkingLot", "removeParkingLot", "analyseUsage", "setPrice"],\n    "User": ["estimateCharge", "confirmEntry", "payCharges", "getTotalCharge"],\n    "ParkingLotOwner": ["setPrices", "allowDynamicPricing"],\n    "ParkingLot": [],\n    "PaymentGateway": ["processPayment"]\n  }\n}'

Identified Operations: {'Administrator': ['addParkingLot', 'removeParkingLot', 'analyseUsage', 'setPrice'], 'User': ['estimateCharge', 'confirmEntry', 'payCharges', 'getTotalCharge'], 'ParkingLotOwner': ['setPrices', 'allowDynamicPricing'], 'ParkingLot': [], 'PaymentGateway': ['processPayment']}


parking_lot_output

Raw Response from LLaMA 3.2:
 {
  "operations": {
    "Administrator": ["addParkingLot", "removeParkingLot", "analyseUsage", "setPrice"],
    "User": ["estimateCharge", "confirmEntry", "payCharges", "getTotalCharge"],
    "ParkingLotOwner": ["setPrices", "allowDynamicPricing"],
    "ParkingLot": [],
    "PaymentGateway": ["processPayment"]
  }
}
Cleaned JSON:
 '{\n  "operations": {\n    "Administrator": ["addParkingLot", "removeParkingLot", "analyseUsage", "setPrice"],\n    "User": ["estimateCharge", "confirmEntry", "payCharges", "getTotalCharge"],\n    "ParkingLotOwner": ["setPrices", "allowDynamicPricing"],\n    "ParkingLot": [],\n    "PaymentGateway": ["processPayment"]\n  }\n}'

Identified Operations: {'Administrator': ['addParkingLot', 'removeParkingLot', 'analyseUsage', 'setPrice'], 'User': ['estimateCharge', 'confirmEntry', 'payCharges', 'getTotalCharge'], 'ParkingLotOwner': ['setPrices', 'allowDynamicPricing'], 'ParkingLot': [], 'PaymentGateway': ['processPayment']}

metro_station_output

Raw Response from LLaMA 3.2:
 {
  "operations": {
    "MetroStation": ["issueTicket", "issueMetroPass", "issueTimeCard"],
    "TicketDistributor": ["selectTripType", "selectQuantity", "choosePaymentMethod", "printTickets", "handleTransaction", "displayErrorMessage"],
    "Traveller": ["selectTripType", "selectQuantity", "issueMetroPass", "issueTimeCard"],
    "Passenger": ["issueMetroPass", "issueTimeCard"],
    "Child": ["issueTicket", "choosePaymentMethod"],
    "Adult": ["issueTicket", "choosePaymentMethod"],
    "MetroPass": ["validateMetroPass", "displayErrorMessage"],
    "TimeCard": ["validateTimeCard", "displayErrorMessage"],
    "Transaction": ["handleTransaction", "displayErrorMessage"],
    "User": ["login", "logout", "displayErrorMessage"],
    "Machine": ["readMetroPass", "readTimeCard", "displayErrorMessage"],
    "TouchScreen": ["displayInterface", "handleInput", "displayErrorMessage"],
    "CurrencyNote": ["validateCurrencyNote", "displayErrorMessage"]
  }
}
Cleaned JSON:
 '{\n  "operations": {\n    "MetroStation": ["issueTicket", "issueMetroPass", "issueTimeCard"],\n    "TicketDistributor": ["selectTripType", "selectQuantity", "choosePaymentMethod", "printTickets", "handleTransaction", "displayErrorMessage"],\n    "Traveller": ["selectTripType", "selectQuantity", "issueMetroPass", "issueTimeCard"],\n    "Passenger": ["issueMetroPass", "issueTimeCard"],\n    "Child": ["issueTicket", "choosePaymentMethod"],\n    "Adult": ["issueTicket", "choosePaymentMethod"],\n    "MetroPass": ["validateMetroPass", "displayErrorMessage"],\n    "TimeCard": ["validateTimeCard", "displayErrorMessage"],\n    "Transaction": ["handleTransaction", "displayErrorMessage"],\n    "User": ["login", "logout", "displayErrorMessage"],\n    "Machine": ["readMetroPass", "readTimeCard", "displayErrorMessage"],\n    "TouchScreen": ["displayInterface", "handleInput", "displayErrorMessage"],\n    "CurrencyNote": ["validateCurrencyNote", "displayErrorMessage"]\n  }\n}'

Identified Operations: {'MetroStation': ['issueTicket', 'issueMetroPass', 'issueTimeCard'], 'TicketDistributor': ['selectTripType', 'selectQuantity', 'choosePaymentMethod', 'printTickets', 'handleTransaction', 'displayErrorMessage'], 'Traveller': ['selectTripType', 'selectQuantity', 'issueMetroPass', 'issueTimeCard'], 'Passenger': ['issueMetroPass', 'issueTimeCard'], 'Child': ['issueTicket', 'choosePaymentMethod'], 'Adult': ['issueTicket', 'choosePaymentMethod'], 'MetroPass': ['validateMetroPass', 'displayErrorMessage'], 'TimeCard': ['validateTimeCard', 'displayErrorMessage'], 'Transaction': ['handleTransaction', 'displayErrorMessage'], 'User': ['login', 'logout', 'displayErrorMessage'], 'Machine': ['readMetroPass', 'readTimeCard', 'displayErrorMessage'], 'TouchScreen': ['displayInterface', 'handleInput', 'displayErrorMessage'], 'CurrencyNote': ['validateCurrencyNote', 'displayErrorMessage']}

### Identify Relationships

In [23]:
import ollama
import json
import re

def identify_relationships(problem_statement, classes, attributes):
    """
    Identifies relationships between extracted classes using Ollama LLaMA 3.2.
    Extracts Association, Generalization (Inheritance), Aggregation, and Composition.
    """
    prompt = f"""
    You are an expert software analyst. Your task is to identify **all types of relationships** between the given classes based on the problem statement.

    **Types of Relationships:**
    1️⃣ **Association** - One class interacts with another. Example: "User borrows Book".
    2️⃣ **Generalization (Inheritance)** - One class is a specialized form of another. Example: "Admin is a subclass of User".
    3️⃣ **Aggregation** - One class is made up of another, but they have independent lifecycles. Example: "Library has Books, but Books exist independently".
    4️⃣ **Composition** - A stronger form of Aggregation, where the part **cannot exist** without the whole. Example: "House has Rooms, Rooms cannot exist without a House".

    **Step-by-Step Process:**
    
    ### **Rules:**
    ✅ Identify the **verbs** and **context clues** in the problem statement to find relationships.  
    ✅ Classify the relationship into one of the four types above.  
    ✅ Include **multiplicities** (`"1"`, `"many"`, `"0..1"`) where applicable.  
    ✅ **Use single verbs** or short phrases (e.g., "creates", "processes", "handles") for descriptions.  
    ✅ Ensure the output follows **strict JSON format** as shown below:  

      ```json
      {{
          "relationships": [
            {{'source': 'TicketDistributor', 'target': 'Ticket', 'type': 'Aggregation', 'description': 'Issues', 'multiplicity': 'many'}}, 
            {{'source': 'TicketDistributor', 'target': 'Transaction', 'type': 'Aggregation', 'description': 'Handles', 'multiplicity': 'many'}}, 
            {{'source': 'TicketDistributor', 'target': 'DisplayInterface', 'type': 'Composition', 'description': 'Uses', 'multiplicity': '1'}}, 
            {{'source': 'TicketDistributor', 'target': 'TouchScreen', 'type': 'Composition', 'description': 'Uses', 'multiplicity': '1'}}, 
            {{'source': 'Transaction', 'target': 'PaymentMethod', 'type': 'Association', 'description': 'Uses', 'multiplicity': '1'}}, 
            {{'source': 'Ticket', 'target': 'Passenger', 'type': 'Association', 'description': 'Issued to', 'multiplicity': '1'}}, 
            {{'source': 'Error', 'target': 'ErrorMessage', 'type': 'Composition', 'description': 'Displays', 'multiplicity': '1'}}
        ]
      }}
      ```

    **Classes Identified:**
    {classes}

    **Attributes Identified:**
    {attributes}

    **Problem Statement:**
    {problem_statement}

    Now extract **all types of relationships** between the classes.
    **Do NOT include any explanation — output JSON only.**
    """

    # Generate response using Ollama LLaMA 3.2
    response = ollama.chat(
        model="llama3.2",
        messages=[{"role": "user", "content": prompt}],
        options={
            "temperature": 0,
            "top_p": 1,
            "top_k": 1
        }
    )

    # ✅ Debugging: Print raw response
    response_text = response['message']['content'].strip()
    print("Raw Response from LLaMA 3.2:\n", response_text)

    # ✅ Extract JSON part only using regex
    json_match = re.search(r"\{.*\}", response_text, re.DOTALL)
    if json_match:
        cleaned_text = json_match.group(0).strip()
    else:
        print("Error: No valid JSON found in response.")
        return {}

    # ✅ Additional Debugging: Print cleaned text before parsing
    print("Cleaned JSON:\n", repr(cleaned_text))

    # ✅ Check if cleaned_text is empty
    if not cleaned_text:
        print("Error: Cleaned JSON is empty. Cannot parse.")
        return {}

    # ✅ Parse JSON safely
    try:
        output = json.loads(cleaned_text)  # Convert string to JSON
        return output.get("relationships", {})  
    except json.JSONDecodeError as e:
        print("JSON parsing error:", str(e))
        return {}


identified_relationships = identify_relationships(problem_statement, identified_classes, identified_attributes)

print("\nRelationships Identified:\n", identified_relationships)


Raw Response from LLaMA 3.2:
 {
  "relationships": [
    {'source': 'MetroStation', 'target': 'TicketDistributor', 'type': 'Aggregation', 'description': 'Has', 'multiplicity': '1'},
    {'source': 'TicketDistributor', 'target': 'Traveller', 'type': 'Association', 'description': 'Issues to', 'multiplicity': 'many'},
    {'source': 'TicketDistributor', 'target': 'MetroPass', 'type': 'Aggregation', 'description': 'Issues', 'multiplicity': 'many'},
    {'source': 'TicketDistributor', 'target': 'TimeCard', 'type': 'Aggregation', 'description': 'Issues', 'multiplicity': 'many'},
    {'source': 'Traveller', 'target': 'Child', 'type': 'Association', 'description': 'Is a type of', 'multiplicity': 'many'},
    {'source': 'Traveller', 'target': 'Adult', 'type': 'Association', 'description': 'Is a type of', 'multiplicity': 'many'},
    {'source': 'TicketDistributor', 'target': 'Transaction', 'type': 'Aggregation', 'description': 'Handles', 'multiplicity': 'many'},
    {'source': 'Transaction', 'ta

In [24]:
identified_relationships =  [
    {'source': 'MetroStation', 'target': 'TicketDistributor', 'type': 'Aggregation', 'description': 'Has', 'multiplicity': '1'},
    {'source': 'MetroStation', 'target': 'Traveller', 'type': 'Association', 'description': 'Has', 'multiplicity': 'many'},
    {'source': 'TicketDistributor', 'target': 'Passenger', 'type': 'Aggregation', 'description': 'Issues', 'multiplicity': '1'},
    {'source': 'TicketDistributor', 'target': 'Child', 'type': 'Association', 'description': 'Is a type of', 'multiplicity': 'many'},
    {'source': 'TicketDistributor', 'target': 'Adult', 'type': 'Association', 'description': 'Is a type of', 'multiplicity': 'many'},
    {'source': 'TicketDistributor', 'target': 'MetroPass', 'type': 'Aggregation', 'description': 'Issues', 'multiplicity': '1'},
    {'source': 'TicketDistributor', 'target': 'TimeCard', 'type': 'Aggregation', 'description': 'Issues', 'multiplicity': '1'},
    {'source': 'Transaction', 'target': 'PaymentMethod', 'type': 'Association', 'description': 'Uses', 'multiplicity': 'many'},
    {'source': 'TicketDistributor', 'target': 'DisplayInterface', 'type': 'Composition', 'description': 'Uses', 'multiplicity': '1'},
    {'source': 'TicketDistributor', 'target': 'TouchScreen', 'type': 'Composition', 'description': 'Uses', 'multiplicity': '1'},
    {'source': 'Transaction', 'target': 'User', 'type': 'Association', 'description': 'Issued to', 'multiplicity': 'many'},
    {'source': 'TicketDistributor', 'target': 'Machine', 'type': 'Aggregation', 'description': 'Uses', 'multiplicity': '1'},
    {'source': 'Transaction', 'target': 'CurrencyNote', 'type': 'Association', 'description': 'Uses', 'multiplicity': 'many'},
    {'source': 'TicketDistributor', 'target': 'Error', 'type': 'Composition', 'description': 'Handles', 'multiplicity': '1'},
    {'source': 'Transaction', 'target': 'ErrorMessage', 'type': 'Composition', 'description': 'Displays', 'multiplicity': '1'}
  ]

In [31]:
identified_relationships = [
    {'source': 'ParkingLotOwner', 'target': 'Administrator', 'type': 'Generalization', 'description': 'is a subclass of', 'multiplicity': '1'},
    {'source': 'ParkingLotOwner', 'target': 'User', 'type': 'Association', 'description': 'owns', 'multiplicity': 'many'},
    {'source': 'ParkingLotOwner', 'target': 'ParkingLot', 'type': 'Aggregation', 'description': 'has', 'multiplicity': '1..*'},
    {'source': 'ParkingLot', 'target': 'User', 'type': 'Association', 'description': 'is used by', 'multiplicity': 'many'},
    {'source': 'PaymentGateway', 'target': 'User', 'type': 'Association', 'description': 'used by', 'multiplicity': '1'},
    {'source': 'Administrator', 'target': 'ParkingLotOwner', 'type': 'Aggregation', 'description': 'manages', 'multiplicity': '0..1'},
    {'source': 'User', 'target': 'PaymentGateway', 'type': 'Association', 'description': 'uses', 'multiplicity': 'many'},
    {'source': 'ParkingLotOwner', 'target': 'Error', 'type': 'Composition', 'description': 'can create', 'multiplicity': '1'}
  ]

### Gnerate plantUML script

In [32]:
import os
import json

def generate_plantuml(classes, attributes,operations, relationships, output_file="parking_lot.puml"):
    """
    Generates a PlantUML class diagram from identified classes, attributes, and relationships.
    """
    plantuml_code = "@startuml\n\n"

     # === Define Classes, Attributes, and Operations ===
    for cls in classes:
        plantuml_code += f"class {cls} {{\n"
        
        # Add attributes if available
        if cls in attributes:
            for attr in attributes[cls]:
                plantuml_code += f"  + {attr}\n"  # Public attribute

        # Add operations if available
        if cls in operations:
            for op in operations[cls]:
                plantuml_code += f"  + {op}()\n"  # Public method

        plantuml_code += "}\n\n"

    # === Convert Multiplicity to UML format ===
    def convert_multiplicity(multiplicity):
        if multiplicity == "many":
            return '"*"'
        elif multiplicity == "1":
            return '"1"'
        elif multiplicity == "0..1":
            return '"0..1"'
        else:
            return '""'  # Default for no multiplicity

    # === Define Relationships ===
    for rel in relationships:
        source = rel["source"]
        target = rel["target"]
        rel_type = rel["type"]
        description = rel["description"]
        multiplicity = convert_multiplicity(rel.get("multiplicity", ""))

        if rel_type == "Association":
            plantuml_code += f'{source} {multiplicity} --> {multiplicity} {target} : {description}\n'
        elif rel_type == "Generalization":
            plantuml_code += f"{source} <|-- {target} : {description}\n"
        elif rel_type == "Aggregation":
            plantuml_code += f'{source} {multiplicity} o-- {multiplicity} {target} : {description}\n'
        elif rel_type == "Composition":
            plantuml_code += f'{source} {multiplicity} *-- {multiplicity} {target} : {description}\n'


    plantuml_code += "\n@enduml"

    # ✅ Save to file
    with open(output_file, "w") as file:
        file.write(plantuml_code)

    print(f"PlantUML file '{output_file}' generated successfully.")


# ✅ Debugging Check
print("Classes Extracted:", identified_classes)
print("Attributes Extracted:", identified_attributes)
print("Operations Extracted:", identified_operations)
print("Relationships Extracted:", identified_relationships)

# ✅ Generate UML
generate_plantuml(identified_classes, identified_attributes, identified_operations , identified_relationships)

# 🖥️ To render, use: `plantuml class_diagram.puml` in the terminal


Classes Extracted: ['Administrator', 'User', 'ParkingLotOwner', 'ParkingLot', 'PaymentGateway']
Attributes Extracted: {'Administrator': ['ID', 'Type', 'Status'], 'User': ['ID', 'Type', 'Status'], 'ParkingLotOwner': ['Name', 'ID', 'Type', 'Status'], 'ParkingLot': ['Name', 'ID', 'Type', 'Status'], 'PaymentGateway': ['Name', 'ID', 'Type', 'Status']}
Operations Extracted: {'Administrator': ['addParkingLot', 'removeParkingLot', 'analyseUsage', 'setPrice'], 'User': ['estimateCharge', 'confirmEntry', 'payCharges', 'getTotalCharge'], 'ParkingLotOwner': ['setPrices', 'allowDynamicPricing'], 'ParkingLot': [], 'PaymentGateway': ['processPayment']}
Relationships Extracted: [{'source': 'ParkingLotOwner', 'target': 'Administrator', 'type': 'Generalization', 'description': 'is a subclass of', 'multiplicity': '1'}, {'source': 'ParkingLotOwner', 'target': 'User', 'type': 'Association', 'description': 'owns', 'multiplicity': 'many'}, {'source': 'ParkingLotOwner', 'target': 'ParkingLot', 'type': 'Aggrega

### Debugging

In [26]:
import ollama
import json
import re

def identify_relationships(problem_statement, classes, attributes):
    """
    Identifies relationships between extracted classes using Ollama LLaMA 3.2.
    Extracts Association, Generalization (Inheritance), Aggregation, and Composition.
    """
    prompt = f"""
    You are an expert software analyst. Your task is to identify **all types of relationships** between the given classes based on the problem statement.

    ### **Types of Relationships:**
    1️⃣ **Association** - One class interacts with another. Example: "User borrows Book".
    2️⃣ **Generalization (Inheritance)** - One class is a specialized form of another. Example: "Admin is a subclass of User".
    3️⃣ **Aggregation** - One class is made up of another, but they have independent lifecycles. Example: "Library has Books, but Books exist independently".
    4️⃣ **Composition** - A stronger form of Aggregation, where the part **cannot exist** without the whole. Example: "House has Rooms, Rooms cannot exist without a House".

    ### **Rules:**
    ✅ Identify the **verbs** and **context clues** in the problem statement to find relationships.  
    ✅ Classify the relationship into one of the four types above.  
    ✅ **Use clear operation-style descriptions** instead of full sentences.  
    ✅ Try to cover **all identified classes** and **attributes** in the relationships.  

    ### **Follow this Step-by-Step Approach:**
    1. Carefully analyze the problem statement to extract interaction patterns between classes.
    2. Identify the type of relationship (Association, Generalization, Aggregation, or Composition).
    3. Formulate a short, clear description using an **operation-style** phrase like "issues", "contains", "inherits", etc.
    4. Ensure that every identified class is considered for possible relationships.

    ### **Output Format:**
    Output **ONLY valid JSON** without any explanation:
    ```json
    {{
        "relationships": [
            {{"source": "ClassA", "target": "ClassB", "type": "Association", "description": "issues"}},
            {{"source": "ClassC", "target": "ClassD", "type": "Aggregation", "description": "contains"}},
            {{"source": "ClassE", "target": "ClassF", "type": "Generalization", "description": "inherits"}}
        ]
    }}
    ```

    ### **Classes Identified:**
    {classes}

    ### **Attributes Identified:**
    {attributes}

    ### **Problem Statement:**
    {problem_statement}

    Now extract **all types of relationships** between the classes.  
    **Do NOT include any explanation — output JSON only.**
    """

    # Generate response using Ollama LLaMA 3.2
    response = ollama.chat(
        model="llama3.2",
        messages=[{"role": "user", "content": prompt}],
        options={
            "temperature": 0,
            "top_p": 1,
            "top_k": 1
        }
    )

    # ✅ Debugging: Print raw response
    response_text = response['message']['content'].strip()
    print("Raw Response from LLaMA 3.2:\n", response_text)

    # ✅ Extract JSON part only using regex
    json_match = re.search(r"\{.*\}", response_text, re.DOTALL)
    if json_match:
        cleaned_text = json_match.group(0).strip()
    else:
        print("Error: No valid JSON found in response.")
        return {}

    # ✅ Additional Debugging: Print cleaned text before parsing
    print("Cleaned JSON:\n", repr(cleaned_text))

    # ✅ Check if cleaned_text is empty
    if not cleaned_text:
        print("Error: Cleaned JSON is empty. Cannot parse.")
        return {}

    # ✅ Parse JSON safely
    try:
        output = json.loads(cleaned_text)  # Convert string to JSON
        return output.get("relationships", [])  # Extract relationships list
    except json.JSONDecodeError as e:
        print("JSON parsing error:", str(e))
        return {}


# Example usage
identified_relationships = identify_relationships(problem_statement, identified_classes, identified_attributes)

print("\nRelationships Identified:\n", identified_relationships)


Raw Response from LLaMA 3.2:
 {
  "relationships": [
    {"source": "MetroStation", "target": "TicketDistributor", "type": "Association", "description": "issues"},
    {"source": "Traveller", "target": "Passenger", "type": "Aggregation", "description": "contains"},
    {"source": "User", "target": "Transaction", "type": "Association", "description": "issues"},
    {"source": "Machine", "target": "DisplayInterface", "type": "Aggregation", "description": "contains"},
    {"source": "PaymentMethod", "target": "Smartcard", "type": "Composition", "description": "is-a"},
    {"source": "MetroPass", "target": "TimeCard", "type": "Aggregation", "description": "contains"},
    {"source": "TicketDistributor", "target": "User", "type": "Association", "description": "issues"},
    {"source": "Traveller", "target": "Child", "type": "Composition", "description": "is-a"},
    {"source": "Traveller", "target": "Adult", "type": "Composition", "description": "is-a"},
    {"source": "MetroStation", "targ